In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/homework-1-dataset/sample_submission.csv
/kaggle/input/homework-1-dataset/data_description.txt
/kaggle/input/homework-1-dataset/train.csv
/kaggle/input/homework-1-dataset/test.csv


In [78]:
X_train = pd.read_csv('/kaggle/input/homework-1-dataset/train.csv')
X_test = pd.read_csv('/kaggle/input/homework-1-dataset/test.csv')

In [79]:
X_test.shape

(1459, 80)

In [80]:
!pip install mlflow dagshub

In [81]:
import mlflow
logged_model = 'runs:/4b0c4220383f4118bdcee4df8152bb8e/Ridge'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(pd.DataFrame(data))

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

# dropping null values
null_ratio = X_train.isna().mean()
too_much_null_columns = null_ratio[null_ratio >= 0.5]
X_train.drop(columns = too_much_null_columns.index, inplace = True)
X_test.drop(columns = too_much_null_columns.index, inplace = True)

# filling numbers with average value
for col in X_train.select_dtypes(include=['number']):
    change_null_to_this = X_train[col].mean()
    X_train.fillna(change_null_to_this, inplace=True)
    X_test.fillna(change_null_to_this, inplace=True)

# filling categoricals with mode value
for col in X_train.select_dtypes(include=['object']):
    change_null_to_this = X_train[col].mode()
    X_train.fillna(change_null_to_this, inplace=True)
    X_test.fillna(change_null_to_this, inplace=True)

# handling categorical values
for col in X_train.select_dtypes(include=['object']):
    encoder = OrdinalEncoder()
    
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)
    
    X_train[col] = encoder.fit_transform(X_train[[col]])
    X_test[col] = encoder.fit_transform(X_test[[col]])

In [83]:
X_test.shape

(1459, 75)

In [84]:
predictions = loaded_model.predict(X_test)

submission_df = pd.DataFrame({
    'Id': X_test['Id'],
    'SalePrice': predictions
})

submission_df.to_csv("submission.csv", index=False)

submission_df

,Id,SalePrice
0,1461,74990.029253
1,1462,127247.221187
2,1463,137819.936230
3,1464,161175.019065
4,1465,159137.571243
...,...,...
1454,2915,41154.069897
1455,2916,27183.199547
1456,2917,126448.436604
1457,2918,87749.177379


In [85]:
Y_train = X_train.pop('SalePrice')
prob_train = loaded_model.predict(X_train)

In [86]:
from sklearn.metrics import *

print("Train:")
r2 = r2_score(Y_train, prob_train)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_train, prob_train, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_train, prob_train)
print(f"MAE score: {mae}")


Train:
R2 score: 0.8458684363000049
RMSE score: 31178.12031834453
MAE score: 19132.342368197435
